<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/ALBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ================================================================
# ALBA-lite protocol (educational version)
# ------------------------------------------------
# Variables explained:
# - A_land_tot (Mkm²): Total continental area on Earth (140–160 Mkm²).
# - share_NH (0–1): Fraction of land placed in the Northern Hemisphere.
# - scenario: Distribution pattern of land within latitude bands.
#              Options = "Uniform", "Equatorial", "Mid-lat", "Polar".
# - spread_deg (°): Concentration width (smaller = continents tightly clustered,
#                   larger = continents more spread across bands).
# - aL_0_10, aL_10_35, aL_35_65, aL_65_90: Land surface albedo in each latitude band.
# - aO_0_10, aO_10_35, aO_35_65, aO_65_90: Ocean surface albedo in each latitude band.
# - S0 (W m⁻²): Solar constant (default ≈1361 W m⁻²).
# - conv_mode: Conversion mode from forcing to ΔT.
#              "Planck-only" = ΔT from Planck feedback,
#              "Fixed-λ" = user-specified constant λ.
# - lambda_fixed (K per W m⁻²): Fixed climate sensitivity factor
#                               used if conv_mode="Fixed-λ".
#
# Outputs:
# - RWT_N, RWT_S: Hemispheric reflectivity (dimensionless).
# - ΔRWT: Hemispheric reflectivity asymmetry.
# - F_comp (W m⁻²): Compensating radiative forcing from ΔRWT.
# - ΔT (K): Estimated thermal response.
# - Four plots: (1) Band albedo NH vs SH, (2) RWT hemispheric comparison,
#               (3) F_comp forcing, (4) ΔT response.
# ================================================================

import numpy as np
import math
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

# Costanti geometriche/fisiche
R_earth = 6371e3
A_earth = 4*math.pi*R_earth**2
A_earth_Mkm2 = A_earth/1e12
sigma_SB = 5.670374419e-8

# Bande climatiche emisferiche
bands = [(0,10),(10,35),(35,65),(65,90)]
band_labels = ["0–10","10–35","35–65","65–90"]
band_midlat = np.array([5.0, 22.5, 50.0, 77.5])  # gradi (centri approssimativi)

def hemi_band_area_Mkm2(phi1_deg, phi2_deg):
    phi1 = math.radians(phi1_deg); phi2 = math.radians(phi2_deg)
    return (2*math.pi*R_earth**2 * (math.sin(phi2) - math.sin(phi1)))/1e12

A_hemi = np.array([hemi_band_area_Mkm2(b[0], b[1]) for b in bands])  # Mkm^2 per banda/emi

def scenario_weights(scenario:str, spread_deg:float):
    """Restituisce pesi non normalizzati per banda (proporzionali all'area * kernel) per una emisfera."""
    # base: proporzionale all'area (Uniform)
    base = A_hemi.copy().astype(float)

    if scenario == "Uniform":
        return base

    # kernel gaussiano centrato su una latitudine target, moltiplicato per l'area
    if scenario == "Equatorial":
        center = 5.0
    elif scenario == "Mid-lat":
        center = 50.0
    elif scenario == "Polar":
        center = 77.5
    else:
        center = 50.0

    sd = max(1.0, float(spread_deg))
    kernel = np.exp(-0.5*((band_midlat - center)/sd)**2)
    return base * kernel

def alba_lite(
    # Superficie terre e sbilanciamento emisferico
    A_land_tot=150.0, share_NH=0.60,
    # Scenario e concentrazione
    scenario="Mid-lat", spread_deg=20.0,
    # Albedo terra per banda
    aL_0_10=0.15, aL_10_35=0.18, aL_35_65=0.25, aL_65_90=0.40,
    # Albedo oceano per banda
    aO_0_10=0.06, aO_10_35=0.07, aO_35_65=0.08, aO_65_90=0.10,
    # Fisica radiativa
    S0=1361.0, conv_mode="Planck-only", lambda_fixed=0.60
):
    # 1) Terre totali e riparto emisferico
    A_land_tot = float(np.clip(A_land_tot, 140.0, 160.0))
    share_NH = float(np.clip(share_NH, 0.0, 1.0))
    A_land_N = A_land_tot * share_NH
    A_land_S = A_land_tot - A_land_N

    # 2) Pesi scenario → frazioni per banda (normalizzate per ciascun emisfero)
    w = scenario_weights(scenario, spread_deg)
    if w.sum() <= 0:
        w = A_hemi.copy()
    frac_bands = w / w.sum()  # somma 1 per emisfero

    # 3) Superficie terra per banda (NH e SH) seguendo lo stesso profilo latitudinale
    L_N = A_land_N * frac_bands
    L_S = A_land_S * frac_bands

    # 4) Albedo superficie per banda per tipo (terra/oceano)
    aL = np.clip(np.array([aL_0_10, aL_10_35, aL_35_65, aL_65_90]), 0.0, 1.0)
    aO = np.clip(np.array([aO_0_10, aO_10_35, aO_35_65, aO_65_90]), 0.0, 1.0)

    # 5) Frazione di terra in ciascuna banda/emi (clamp 0..1 rispetto all'area disponibile)
    fL_N = np.clip(L_N / A_hemi, 0.0, 1.0)
    fL_S = np.clip(L_S / A_hemi, 0.0, 1.0)

    # 6) Albedo superficiale band-media per emisfero: α_S = fL*α_L + (1−fL)*α_O
    aS_N = fL_N*aL + (1.0 - fL_N)*aO
    aS_S = fL_S*aL + (1.0 - fL_S)*aO

    # 7) RWT emisferici (media area-pesata rispetto alla superficie globale)
    #    RWT_emi = sum_b( α_S_emi(banda) * A_banda ) / A_globale
    RWT_N = float(np.sum(aS_N * (A_hemi / A_earth_Mkm2)))
    RWT_S = float(np.sum(aS_S * (A_hemi / A_earth_Mkm2)))
    dRWT = abs(RWT_N - RWT_S)

    # 8) Forzante di “compensazione emisferica”
    F_comp = - dRWT * (S0/4.0)  # W m^-2 (segno convenzionale: più asimmetria → forzante negativa qui)

    # 9) ΔT tramite solo feedback di Planck o tramite λ fisso
    if conv_mode == "Planck-only":
        # temperatura di equilibrio del pianeta con RWT medio “compensato”
        RWT_high = max(RWT_N, RWT_S); RWT_low = min(RWT_N, RWT_S)
        RWT_comp = RWT_high + RWT_low + dRWT  # = 2*RWT_high (come nel suo schema originale)
        Te = ((1.0 - RWT_comp) * S0 / (4.0*sigma_SB))**0.25
        lambda_P = 4.0*sigma_SB*Te**3  # W m^-2 K^-1
        dT = F_comp / lambda_P
    else:
        dT = lambda_fixed * F_comp  # K

    # === Riepilogo numerico
    print("=== ALBA-lite (scenario + sbilanciamento NH/SH) ===")
    print(f"Scenario = {scenario} | spread = ±{spread_deg:.1f}° | Land tot = {A_land_tot:.1f} Mkm²")
    print(f"Land NH = {A_land_N:.1f} Mkm² ({share_NH*100:.1f}%), Land SH = {A_land_S:.1f} Mkm²")
    print(f"RWT_N = {RWT_N:.5f} | RWT_S = {RWT_S:.5f} | ΔRWT = {dRWT:.5f}")
    print(f"F_comp = {F_comp:.3f} W m^-2 | ΔT = {dT:.3f} K  | ΔF→ΔT = {('Planck-only' if conv_mode=='Planck-only' else f'λ fisso={lambda_fixed:.2f} K/(W m^-2)')}")

    # === Grafico 1: albedo per banda (NH vs SH)
    x = np.arange(4); width=0.35
    plt.figure(figsize=(7,4))
    plt.bar(x - width/2, aS_N, width, label='NH')
    plt.bar(x + width/2, aS_S, width, label='SH')
    plt.xticks(x, band_labels)
    plt.ylabel("α_surface (band)")
    plt.title("Albedo superficiale per banda (NH vs SH)")
    plt.legend(); plt.grid(True, axis='y', linestyle=':')
    plt.show()

    # === Grafico 2: RWT emisferici
    plt.figure(figsize=(6,4))
    plt.bar(["RWT_N","RWT_S"], [RWT_N, RWT_S])
    plt.ylabel("RWT (adimensionale)")
    plt.title("RWT per emisfero")
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    # === Grafico 3: Forzante F_comp
    plt.figure(figsize=(6,3.8))
    plt.bar(["F_comp"], [F_comp])
    plt.ylabel("W m$^{-2}$")
    plt.title(f"Forzante emisferica: F_comp = {F_comp:.3f} W m$^{{-2}}$")
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    # === Grafico 4: ΔT
    plt.figure(figsize=(6,3.8))
    plt.bar(["ΔT"], [dT])
    plt.ylabel("K")
    if conv_mode == "Planck-only":
        plt.title(f"Risposta termica (Planck-only): ΔT = {dT:.3f} K")
    else:
        plt.title(f"Risposta termica (λ fisso): ΔT = {dT:.3f} K")
    plt.grid(True, axis='y', linestyle=':')
    plt.show()

    return {"RWT_N":RWT_N, "RWT_S":RWT_S, "ΔRWT":dRWT, "F_comp":F_comp, "ΔT":dT}

# UI interattiva
interact(
    alba_lite,
    A_land_tot=FloatSlider(value=150.0, min=140.0, max=160.0, step=0.5, description="Land tot (Mkm²)"),
    share_NH=FloatSlider(value=0.60, min=0.0, max=1.0, step=0.01, description="NH land share"),
    scenario=Dropdown(options=["Uniform","Equatorial","Mid-lat","Polar"], value="Mid-lat", description="Scenario"),
    spread_deg=FloatSlider(value=20.0, min=5.0, max=45.0, step=1.0, description="Concentration (°)"),
    aL_0_10=FloatSlider(value=0.15, min=0.05, max=0.60, step=0.005, description="α_L 0–10"),
    aL_10_35=FloatSlider(value=0.18, min=0.05, max=0.60, step=0.005, description="α_L 10–35"),
    aL_35_65=FloatSlider(value=0.25, min=0.05, max=0.80, step=0.005, description="α_L 35–65"),
    aL_65_90=FloatSlider(value=0.40, min=0.05, max=0.90, step=0.005, description="α_L 65–90"),
    aO_0_10=FloatSlider(value=0.06, min=0.02, max=0.20, step=0.001, description="α_O 0–10"),
    aO_10_35=FloatSlider(value=0.07, min=0.02, max=0.20, step=0.001, description="α_O 10–35"),
    aO_35_65=FloatSlider(value=0.08, min=0.02, max=0.25, step=0.001, description="α_O 35–65"),
    aO_65_90=FloatSlider(value=0.10, min=0.02, max=0.30, step=0.001, description="α_O 65–90"),
    S0=FloatSlider(value=1361.0, min=1300.0, max=1400.0, step=1.0, description="S0 (W m⁻²)"),
    conv_mode=Dropdown(options=["Planck-only","Fixed-λ"], value="Planck-only", description="ΔF→ΔT"),
    lambda_fixed=FloatSlider(value=0.60, min=0.2, max=1.0, step=0.01, description="λ_fixed K/(W m⁻²)")
);